![](https://uploads-ssl.webflow.com/62233c592d2a1e009d42f46c/6414802c0a2bea367cbc795b_logo-jedha-square.svg)

<h1 style="text-align: left; color:#20a08d; font-size: 40px"><span><strong>Comprendre l'apprentissage par renforcement profond 🧠🧠
</strong></span></h1>

Le Deep reinforcement learning consiste à mélanger l'apprentissage en profondeur avec des techniques classiques d'apprentissage par renforcement. L'aspect clé est que nous sommes confrontés à un environnement avec un nombre infini d'états (parfois même pas énumérables), donc l'idée est de prendre la description de l'environnement comme entrée d'un modèle d'apprentissage en profondeur (il pourrait être exprimé sous forme d'images par exemple ) et le faire correspondre à un espace de représentation que nous utiliserons comme projection de l'environnement pour prendre des décisions.

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Qu'allez-vous apprendre dans ce cours ? 🧐🧐
</strong></span></h2>

- Prédiction (évaluation des politiques)
- Contrôle (trouver la meilleure stratégie en estimant les valeurs des paires d'action d'état)
- L'apprentissage profond (Deep Learning)

L'objectif du modèle est de cartographier des états de l'environnement qui sont similaires à des représentations connues de l'agent afin que l'agent puisse prendre plus simplement les décisions qui lui rapporteront le maximum de récompense cumulée.

Nous utiliserons **l'apprentissage par renforcement profond dans des situations où l'espace d'état n'est pas fini**. Des exemples pourraient être que nous avons des métriques avec des espaces de valeurs continus, comme l'environnement cartpole. Ou vous pourriez penser à un jeu d'arcade comme atari's pong, où nous essayons de comprendre quelles actions entreprendre en fonction des images produites par le jeu.

Comme nous avons beaucoup d'états (un nombre infini), le Q-learning ne peut pas fonctionner, car le Q-learning associe une valeur à chaque paire état-action. Nous allons donc devoir procéder à l'approximation de la fonction. Cela signifie utiliser une fonction pour mapper les états et les actions sur des valeurs, et utiliser cette fonction pour extrapoler la valeur des états observés afin d'estimer les valeurs des états ressemblants non observés.

L'apprentissage par renforcement profond **utilise des réseaux de neurones** pour mapper les états à un espace de représentation spécifique qui tente de produire les valeurs de chaque paire d'action d'état.

Explorons les méthodes de prédiction et de contrôle, c'est-à-dire l'estimation des valeurs d'état et l'estimation des valeurs de paires action-état.

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Prediction 🔮</strong></span></h2>

Approximation de la fonction de valeur (état -> récompense)

S'il y a beaucoup d'états, nous avons besoin de beaucoup d'expérience pour apprendre toutes les valeurs d'états , nous utilisons donc l'approximation de la fonction afin de généraliser la valeur des états observés à des états similaires.

$$
\begin{align*}
v_w (s) & \approx v_{\pi}(s) \\
q_w(s,a) & \approx q_{\pi}(s,a)
\end{align*}
$$

Notre objectif est de mettre à jour le jeu de paramètres $w$ et obtenez une fonction qui peut se généraliser à des états non encore vus.

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>Approximation de la fonction linéaire
</strong></span></h3>

Mappez les états sur un ensemble de nombres réels comme (position de la base, vitesse de la base, angle du bras, vitesse angulaire du bras)

$$
\begin{align*}
x :& \mathcal{S} \rightarrow \mathbb{R}^n \\
& s \rightarrow x(s)
\end{align*}
$$

Ensuite, nous faisons une approximation linéaire des fonctions de valeur :

$$
v_w(s) = w^Tx(s)
$$

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>Coarse Coding (codage grossier)
</strong></span></h3>

Si nous souhaitons mapper notre espace d'états infini sur un espace d'états fini, nous pouvons recourir à une technique appelée **coarse coding** (ou alternativement **tile coding**). Cette technique permet de discrétiser l'espace en le mappant sur une collection de variables binaires dont les valeurs correspondent à :

- 0 : l'état n'appartient pas à cet espace de valeurs
- 1 : l'état appartient à cet espace de valeurs

Cela nous permet d'utiliser les valeurs obtenues à partir des états observés pour influencer les valeurs des états voisins. La précision que vous recherchez et la puissance de calcul dont vous disposez influenceront fortement le type de cartographie que vous ferez.

Voici un exemple de codage grossier pour un espace d'état à deux dimensions :
![coarse coding](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/LEAD/Reinforcement-Learning/coarse_coding.png)

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>Approximation de la fonction différentiable

</strong></span></h3>

$v_w(s)$ est une fonction différentiable avec un ensemble de paramètres $w$ qui pourrait être non linéaire

Vous pouvez également imaginer un cas où les les "features maps" que la fonction de valeur obtient en tant qu'entrées sont apprises et non corrigées (comme si vous essayez d'interpréter une image d'un jeu comme un vecteur de caractéristiques ou quelque chose)

Considérons une fonction $J(w)$, qui jouerait le rôle de la fonction "perte" pour notre problème d'apprentissage par renforcement.

Le gradient de $J$ est

$$
\nabla_w J(w) = (\frac{\partial J(w)}{\partial w_1}, \dots, \frac{\partial J(w)}{\partial w_p})
$$

Notre objectif est de minimiser J. Nous pourrions faire une descente de gradient et nous déplacer dans le sens opposé du gradient

$$
w^{(t+1)}=w^{(t)} - \frac{1}{2}\gamma \nabla_w J(w)
$$

si

$$
J(w) = \mathbb{E}_{S~d}[(v_{\pi}(S) - v_w(S))^2]
$$

L'erreur quadratique attendue entre la politique et sa politique basée sur l'approximation de la fonction de valeur

La descente du gradient ressemble alors à
$$
w^{(t+1)}=w^{(t)} - \gamma \mathbb{E}_{S~d}[v_{\pi}(S) - v_w(S)] \nabla_w v_w(S)
$$

Étant donné qu'en pratique, la valeur attendue réelle peut être inobservable ou intraitable, nous pouvons alors choisir d'utiliser la descente de gradient stochastique en travaillant avec des estimations 

$$
w^{(t+1)}=w^{(t)} - \gamma (G_t - v_w(S_t)) \nabla_w v_w(S_t)
$$

Où nous mettons progressivement à jour le $G_t$ grâce à une méthode de Monte Carlo (où $G_t$ est la récompense cumulée après observation de l'état $S_t$)

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Algorithme de contrôle 🎮</strong></span></h2>

Comme toujours, l'évaluation des valeurs des états ne représente que la moitié du problème d'apprentissage par renforcement. Afin de le résoudre, nous devons savoir quelles actions conduisent aux niveaux de récompense optimaux

- Le Q-learning peut être étendu à l'approximation de fonctions (réseau Q profond - DQN)
- Théorie pas complètement développée
- Le "tracking" (ce qui signifie que la politique peut s'adapter en permanence) est généralement préféré à la convergence vers une politique fixe

Pour la descente de gradient par lots avec l'apprentissage par renforcement :

1. Le réseau de neurones prend l'observation O pour la transformer en une valeur d'action pour chaque action possible, puis nous utilisons une politique epsilon-greedy pour prendre l'action suivante.

2. Ensuite, les mises à jour de poids se déroulent comme suit :
$$
\Delta w \propto (R_{t+1} + \gamma \max_a q_w (S_{t+1},a) - q_w (S_t, A_t)) \nabla_w q_w (S_t, A_t)
$$

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>Algorithme DQN
</strong></span></h3>

1. $O_t \rightarrow q_w$ : Mettre plusieurs "frames" ensemble pour les jeux atari par exemple, donc une observation n'est pas nécessairement un seul état mais une séquence de plusieurs états.

2. Nous utilisons une politique d'exploration (epsilon-greedy)

3. Nous gardons également un tampon de relecture pour stocker des exemples de transitions passées $(S_i, A_i, R_{i+1},S_{i+1})$ Celles-ci seront utilisées pour entraîner le modèle pendant plusieurs itérations jusqu'à ce qu'elles soient remplacées par de nouvelles observations au fil du temps.

4. Nous avons mis en place des paramètres de réseau cible $w^{-}$ qui reste constant sur plusieurs pas de temps. Le réseau cible est utilisé pour sélectionner la meilleure action dans l'état suivant de l'environnement (cela empêche l'apprentissage Q pour sélectionner des actions qui suivent strictement l'évolution des estimations, et évite de surestimer certaines paires de valeurs d'action d'état).

5. La mise à jour des poids Q learning sur $w$ (utilise les données de relecture et le réseau cible)

$$
\Delta w \propto (R_{t+1} + \gamma \max_a q_{w^{-}} (S_{i+1},a) - q_w (S_i, A_i)) \nabla_w q_w (S_i, A_i)
$$

6. Ensuite, nous mettons à jour les paramètres du réseau cible$w_t^{-} \leftarrow w_t$ occasionnellement (tous les 10 000 pas par exemple)

L'utilisation de données de relecture et d'un réseau cible fait ressembler DQN à un apprentissage supervisé.

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Quel rôle jouent les réseaux de neurones profonds dans tout cela ?📈</strong></span></h2>

L'apprentissage en profondeur entre en jeu pour la partie d'approximation de la fonction, que vous traitiez un problème de prédiction (estimer la fonction de valeur pour chaque état donné une certaine politique) ou le problème de contrôle (estimer la fonction de valeur de l'état d'action pour chaque état et action ) le réseau neuronal profond aidera à mapper les états sur des valeurs spécifiques ou des paires de valeurs d'action.

Par exemple:
- Dans le cas de la prédiction (estimation de la fonction de valeur pour chaque état), le modèle de réseau neuronal profond sera une fonction prenant ses entrées dans $\mathbb{R}^n$ $n$ étant la dimensionnalité de l'espace d'état (par exemple, si l'espace d'état est l'écran d'un jeu atari avec une résolution de 200x200 en couleurs, alors la dimension de l'entrée sera $(200,200,3)$, et produisant des sorties dans $\mathbb{R}$ pour les valeurs. Vous pouvez considérer cela comme un réseau neuronal convolutif avec une dernière couche contenant un neurone avec une fonction d'activation linéaire.
- Dans le cas du contrôle (estimation des paires de valeurs d'état d'action), le réseau de neurones a des entrées dans $\mathbb{R}^n$, et sorties en $\mathbb{R}^m$ , où $n$ est la dimensionnalité de l'espace d'entrée, et $m$ le nombre d'actions possibles. Pour l'exemple du jeu atari, il peut s'agir d'un réseau de neurones convolutifs avec une dernière couche dense avec $m$ neurones et activation linéaire.

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Deadly triad 💀</strong></span></h2>

La Deadly triad (triade mortelle) est une combinaison de "raccourcis" utilisés dans l'apprentissage par renforcement pour accélérer ou simplifier le processus d'entrainement qui peut empêcher le modèle de converger du tout vers la politique optimale.

Ces trois éléments sont :

- **bootstraping (apprentissage à partir d'estimations)** : qui est l'aspect central du Q-learning
- **approximation de fonction** : qui est au cœur de l'apprentissage par renforcement profond, puisque nous avons affaire à des espaces d'états observables qui sont infinis
- **apprentissage hors politique** : qui est au cœur du Q-learning, puisque nous utilisons une politique epsilon-greedy afin d'apprendre la politique optimale.

## Ressources 📚📚

* [The reference book for all things reinforcement learning (from which the figures were taken)](https://full-stack-assets.s3.eu-west-3.amazonaws.com/references/reinforcement_learning/SuttonBartoIPRLBook2ndEd.pdf)
* [The Deepmind (alpha go creators) reinforcement learning lectures](https://www.deepmind.com/learning-resources/reinforcement-learning-lecture-series-2021)